# Quantum Simulation of a free Schrödinger Wave
This implementation is based upon the paper **"Simulating Quantum Mechanics on a Quantum Computer"** attached in /research

In [197]:
from qiskit import QuantumCircuit as QC
from qiskit import transpile
from qiskit.circuit.library import iSwapGate
import numpy as np
from itertools import product
from qiskit_aer import QasmSimulator

## 1. State Preparation
Using a recursive controlled rotation algorithm, we match the amplitudes of each qubit state with the given discretized values of the wave equation

###  1.1 Mock 3 values Wave amplitudes

In [198]:
psi = np.array([0.5, 0.5 + 0.7j, 0.5], dtype=complex)# complex wavefunction
psi /= np.linalg.norm(psi)  # Normalize the wavefunction
print(psi)

[0.44901326+0.j         0.44901326+0.62861856j 0.44901326+0.j        ]


### 1.2 Prepare Quantum Circuit with 3 qubits and 3 classical bits as registers
As discussed in the paper, each qubit represents a discrete position.

So n positions require n qubits, not the most optimal solution.

In [199]:
# qc = QC(len(psi), len(psi))

### 1.3 Rotating into correct state vector

#### 1.3.1 - Probability Tree

```plaintext
Amplitude Tree for |ψ> = c1|100> + c2|010> + c3|001>

Start: |000>  (amplitude = 1)
│
├─ R_y(θ1) on q0
│   ├─ |100> :  sin(θ1/2) = c1
│   └─ |000> :  cos(θ1/2) = √(1 - c1²)
│
├─ Controlled R_y(θ2) on q1  (control q0 = 0)
│   ├─ |010> :  √(1 - c1²) * sin(θ2/2) = c2
│   └─ |000> :  √(1 - c1²) * cos(θ2/2) = √(1 - c1² - c2²)
│
└─ Controlled R_y(θ3) on q2  (controls q0 = 0, q1 = 0)
    ├─ |001> :  √(1 - c1² - c2²) * sin(θ3/2) = c3
    └─ |000> :  √(1 - c1² - c2²) * cos(θ3/2) = 0
```

Start: |000>

1. Rotate qubit 0: 
$|\psi\rangle' = cos(\theta_1/2)*|000\rangle+sin(\theta_1/2)*|100\rangle$

```plaintext
    ry(theta, qubit, label=None)
```

In [200]:
# theta1 = 2*np.arcsin(psi[0]).real
# qc.ry(theta1, 0)

2. Controlled rotation of qubit 1, ctrl being qubit 0:

```plaintext
    cry(theta, control_qubit, target_qubit, label=None, ctrl_state=None)
```

In [201]:
# theta2 = 2*np.arcsin(psi[1]/np.sqrt(1-psi[0]**2)).real
# qc.cry(theta2, 0, 1, ctrl_state=0)

3. Multi controlled rotation of qubit 2, ctrl being qubits 0 and 1

Visualization:
```plaintext
| Original (q0, q1) | After first Xs | Controlled gate acts? | After last Xs (restored) |
|--------------------|----------------|------------------------|---------------------------|
| 00                 | 11             | ✅ Yes                 | 00                        |
| 01                 | 10             | ❌ No                  | 01                        |
| 10                 | 01             | ❌ No                  | 10                        |
| 11                 | 00             | ❌ No                  | 11                        |

```

In [202]:
# theta3 = 2*np.arcsin(psi[2]/np.sqrt(1-psi[0]**2-psi[1]**2)).real
# qc.x(0)
# qc.x(1)
# qc.mcry(theta3, [0, 1], 2)
# qc.x(0)
# qc.x(1)

qubit 4

In [203]:
# theta4 = 2*np.arcsin(psi[3]/np.sqrt(1-psi[0]**2-psi[1]**2-psi[2]**2)).real
# qc.x(0)
# qc.x(1)
# qc.x(2)
# qc.mcry(theta4, [0, 1, 2], 3)
# qc.x(0)
# qc.x(1)
# qc.x(2)

qubit 5

In [204]:
# theta5 = 2*np.arcsin(psi[4]/np.sqrt(1-psi[0]**2-psi[1]**2-psi[2]**2-psi[3]**2)).real
# qc.x(0)
# qc.x(1)
# qc.x(2)
# qc.x(3)
# qc.mcry(theta5, [0, 1, 2, 3], 4)
# qc.x(0)
# qc.x(1)
# qc.x(2)
# qc.x(3)

State preparation algo:

In [205]:
def flip_qubits(qc : QC, qubits : list[int]) -> QC:
    for q in qubits:
        print(f"Flipping q{q}")
        qc.x(q)
    return qc

def state_preparation(qc : QC) -> QC:
    controls = [0]
    
    # Init first qubit
    theta1 = 2*np.arcsin(np.abs(psi[0])).real
    qc.ry(theta1, 0)

    phi1 = np.angle(psi[0]) 
    qc.rz(phi1, 0) 

    # # Prepare second qubit
    theta2 = 2*np.arcsin(np.abs(psi[1])/np.sqrt(1-psi[0]**2)).real
    qc.cry(theta2, 0, 1, ctrl_state=0)

    phi2 = np.angle(psi[1])
    qc.crz(phi2, 0, 1, ctrl_state=0)

    # Subsequent qubits
    for i in range(1, qc.num_qubits - 1): # i+1 -> target
        controls.append(i)
        print("Controls:", controls)

        theta = 2*np.arcsin(
            psi[i+1]/np.sqrt(1-sum([np.abs(psi[j])**2 for j in range(i+1)]))
            ).real
        
        phi = np.angle(psi[i+1])

        print("Theta-loop:", theta)

        # theta2 = 2*np.arcsin(psi[1]/np.sqrt(1-psi[0]**2)).real
        # print("Theta 2-manual:", theta2)
        # theta3 = 2*np.arcsin(psi[2]/np.sqrt(1-psi[0]**2-psi[1]**2)).real
        # print("Theta 3-manual:", theta3)

        flip_qubits(qc, controls)
        qc.mcry(theta, controls, qc.qubits[i+1])
        qc.mcrz(phi, controls, qc.qubits[i+1])
        # print(f"Applied mcry with theta={theta} on qubit {i+1} with controls {controls}")
        print(f"State prep for qubit {i+1} done.")
        flip_qubits(qc, controls)

    return qc

qc = QC(len(psi), len(psi))
print("Initial circuit: |000>")
qc = state_preparation(qc)

Initial circuit: |000>
Controls: [0, 1]
Theta-loop: 3.1415925747402595
Flipping q0
Flipping q1
State prep for qubit 2 done.
Flipping q0
Flipping q1


## 2. Unitary transformation in free space

$ U(t) = exp(-i*H*\delta t) $, with H being the Hamiltonian and $ \hbar \ = 1 $

In [206]:
a = 1j/np.sqrt(2)
b = 1/np.sqrt(2)
U = np.matrix([[1, 0, 0, 0],
               [0, b, a, 0],
               [0, a, b, 0],
               [0, 0, 0, 1]])

We define:

$$
b = \cos(\theta), \quad a = i \sin(\theta)
$$

The two-qubit state is written as:

$$
|\psi\rangle = c_{00}|00\rangle + c_{01}|01\rangle + c_{10}|10\rangle + c_{11}|11\rangle
$$

After applying \(U\), the coefficients transform as:

$$
\begin{aligned}
c'_{00} &= c_{00},\\[4pt]
\begin{pmatrix}
c'_{01}\\
c'_{10}
\end{pmatrix}
&=
\begin{pmatrix}
\cos\theta & i\sin\theta\\
i\sin\theta & \cos\theta
\end{pmatrix}
\begin{pmatrix}
c_{01}\\
c_{10}
\end{pmatrix},\\[4pt]
c'_{11} &= c_{11}.
\end{aligned}
$$

The corresponding matrix representation (in the basis $|00\rangle, |01\rangle, |10\rangle, |11\rangle$) is:

$$
U =
\left(
\begin{array}{c|cccc}
 & |00\rangle & |01\rangle & |10\rangle & |11\rangle \\ \hline
|00\rangle & 1 & 0 & 0 & 0 \\
|01\rangle & 0 & \cos\theta & i\sin\theta & 0 \\
|10\rangle & 0 & i\sin\theta & \cos\theta & 0 \\
|11\rangle & 0 & 0 & 0 & 1
\end{array}
\right)
$$


In [207]:
def time_transform(circuit: QC, U: np.matrix):
    theta = 2 * np.arctan2(np.abs(a), np.abs(b)) 
    # theta = np.pi/2  # Example fixed angle for demonstration
    n = range(circuit.num_qubits - 1)
    for i in n:
        circuit.rxx(-theta, circuit.qubits[i], circuit.qubits[i+1])
        circuit.ryy(-theta, circuit.qubits[i], circuit.qubits[i+1])
    return circuit

## 3. Time Evolution

In [208]:
for i in range(np.pow(2,10)):
    qc = time_transform(qc, U)

## 4. Quantum Simulator - Measurement

In [209]:
measure_list = []
for i in range(0, qc.num_qubits):
    measure_list.append(i)
qc.measure(measure_list, measure_list)
print(measure_list)

[0, 1, 2]


In [210]:
backend = QasmSimulator(method='statevector')
shots = np.pow(2,20)
job = backend.run(qc, shots=shots)
result = job.result()

counts = result.get_counts()
print(counts)

{'100': 211359, '010': 211256, '001': 625961}


## 5. Map into np array

In [211]:
# --- total number of shots
# shots = sum(counts.values())

# --- all 5-qubit basis states (2^5 = 32)
states = [''.join(bits) for bits in product('01', repeat=qc.num_qubits)]

# --- convert counts → probabilities
pvec = np.array([counts.get(s, 0) / shots for s in states])

# --- make a matching NumPy array of state labels
states_arr = np.array(states)

# --- eliminate zero components
mask = pvec > 0
pvec_nonzero = pvec[mask]
states_nonzero = states_arr[mask]

# --- (optional) compute amplitudes from probabilities
amps_nonzero = np.sqrt(pvec_nonzero)

# --- pretty print results
print("Non-zero states:", states_nonzero)
print("Probabilities:", pvec_nonzero)
print("Amplitudes:", amps_nonzero)

initial_probs = []
for i in range(len(psi)):
    initial_probs.append(abs(psi[i])**2)

print("Initial probabilities:", initial_probs)

# --- optional: combine into a dictionary
# filtered = dict(zip(states_nonzero, pvec_nonzero))
# filtered_amps = dict(zip(states_nonzero, amps_nonzero))
# print("Dictionary (probabilities):", filtered)
# print("Dictionary (amplitudes):", filtered_amps)


Non-zero states: ['001' '010' '100']
Probabilities: [0.59696293 0.20146942 0.20156765]
Amplitudes: [0.77263376 0.44885345 0.44896286]
Initial probabilities: [np.float64(0.20161290322580644), np.float64(0.5967741935483869), np.float64(0.20161290322580644)]


## 6. Dispersion factor Calculator

In [212]:
def dispersion_factor(initial_psi: np.ndarray, final_amps: np.ndarray) -> float:
    """
    Compare initial and final amplitude distributions to quantify dispersion.
    Returns a number in [0, 1]:
        0 = identical distributions (no spread)
        1 = fully different / maximally dispersed
    """
    # --- Normalize both wavefunctions
    p_init = np.abs(initial_psi)**2
    p_init /= p_init.sum()
    
    p_final = np.abs(final_amps)**2
    p_final /= p_final.sum()
    
    # --- Compute cosine similarity of the two probability vectors
    numerator = np.sum(p_init * p_final)
    denom = np.sqrt(np.sum(p_init**2) * np.sum(p_final**2))
    similarity = numerator / denom if denom != 0 else 0.0
    
    # --- Dispersion factor (1 - similarity)
    dispersion = 1 - similarity
    return dispersion

print(f"Dispersion factor: {dispersion_factor(psi, amps_nonzero)}")

Dispersion factor: 0.3572121827160304


## 7. Test with iSwap given theta = pi/6

1. Reinitialize qubit states

In [213]:
# qc_test = QC(3, 3)

# theta1 = 2*np.arcsin(psi[0]).real
# qc_test.ry(theta1, 0)

# theta2 = 2*np.arcsin(psi[1]/np.sqrt(1-psi[0]**2)).real
# qc_test.cry(theta2, 0, 1, ctrl_state=0)

# theta3 = 2*np.arcsin(psi[2]/np.sqrt(1-psi[0]**2-psi[1]**2)).real
# qc_test.x(0)
# qc_test.x(1)
# qc_test.mcry(theta3, [0, 1], 2)
# qc_test.x(0)
# qc_test.x(1)

2. Transform using iSwap gates

    IBM iSwap Docs: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.iSwapGate

In [214]:
# n = range(qc_test.num_qubits - 1)
# for i in range(2):
#     for k in n:
#         qc_test.iswap(k, k+1)

3. Measure

In [215]:
# qc_test.measure([0, 1, 2], [0, 1, 2])

# backend_test = QasmSimulator(method='statevector')
# qc_test = transpile(qc_test, backend=backend_test, basis_gates=['u3', 'cx'])
# job_test = backend.run(qc_test, shots=np.pow(2,20))
# result_test = job_test.result()

# counts_test = result_test.get_counts()
# print(counts)